In [12]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import requests
import pprint
from bs4 import BeautifulSoup
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
client = MongoClient('10.52.51.28', 27017).ourinhos.bitcoin
url = 'https://www.infomoney.com.br/cryptos/cotacoes'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
crypt = soup.find_all(class_ = "ticker-internal")

In [4]:
client.delete_many({})

In [5]:
for coin in crypt:
    titulo = str(coin.find(class_ = "crypto__ticker-item-logo")).split('title')[1].replace('="', '').replace('"/>', '')
    
    if (titulo != 'Conheça mais sobre a Xdex'):
        dh_pesquisa = datetime.datetime.now()
        nm_moeda = coin.find('a').text
        pc_percentual = coin.find(class_ = 'ticker-box-positive').text
        vr_valor = coin.find(class_ = "value").text
 
        insert = {
            'dh_pesquisa': dh_pesquisa.strftime("%Y-%m-%d %H:%M"),
            'nm_moeda': nm_moeda,
            'pc_percentual': float(pc_percentual.replace('%', '').replace('+', '').replace(',', '.')),
            'vr_valor': float(vr_valor.split()[1].replace('%', '').replace('+', '').replace('.', '').replace(',', '.'))}
    
        client.insert_one(insert).inserted_id

In [9]:
moeda = []

client = MongoClient('10.52.51.28', 27017).ourinhos.bitcoin
for document in client.find():
    moeda.append([document['nm_moeda'], document['pc_percentual'], document['vr_valor']])
    #percentual.append(document['pc_percentual'])
    #valor.append(document['vr_valor'])
client.close

Collection(Database(MongoClient(host=['10.52.51.28:27017'], document_class=dict, tz_aware=False, connect=True), 'ourinhos'), 'bitcoin.close')

In [16]:
df = pd.DataFrame(moeda, columns='Nome Percentual Valor'.split())
df.set_index('Nome',inplace=True)
df

,Percentual,Valor
Nome,,
BITCOIN,3.20,15441.00
ETHEREUM,5.25,587.19
RIPPLE,2.16,1.23
EOS,4.91,15.27
LITECOIN,3.27,191.67
BITCOIN CASH,2.79,555.90
TETHER,0.04,3.78
STELLAR,3.11,0.35


In [35]:
vl = [[df['Valor'].sum(), df['Valor'].mean(), df['Valor'].max(), df['Valor'].min()]]
dfvl = pd.DataFrame(vl, columns='Total Média Máximo Mínimo'.split())
dfvl

,Total,Média,Máximo,Mínimo
0,16796.39,2099.54875,15441.0,0.35


In [41]:
df.sort_values(['Valor'], ascending = False)

,Percentual,Valor
Nome,,
BITCOIN,3.20,15441.00
ETHEREUM,5.25,587.19
BITCOIN CASH,2.79,555.90
LITECOIN,3.27,191.67
EOS,4.91,15.27
TETHER,0.04,3.78
RIPPLE,2.16,1.23
STELLAR,3.11,0.35


In [43]:
df.sort_values(['Percentual'], ascending = False)

,Percentual,Valor
Nome,,
ETHEREUM,5.25,587.19
EOS,4.91,15.27
LITECOIN,3.27,191.67
BITCOIN,3.20,15441.00
STELLAR,3.11,0.35
BITCOIN CASH,2.79,555.90
RIPPLE,2.16,1.23
TETHER,0.04,3.78
